<a href="https://colab.research.google.com/github/t-fushihara/FS_blog/blob/main/convert_to_volumebar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
import time

In [ ]:
data = pd.read_csv('bitstampUSD_1-min_data_2012-01-01_to_2020-12-31.csv')
data.head()

In [ ]:
data['datetime'] = pd.to_datetime(data.Timestamp, unit='s')
data = data.set_index('datetime').drop('Timestamp',axis=1)
data = data.dropna(how='any')['2017':].drop('Weighted_Price',axis=1)

In [ ]:
#Volume_barを作成する関数
#入力dfの列は必ず以下の通りの順番にする
#['Open', 'High', 'Low', 'Close', 'Volume_(BTC)', 'Volume_(Currency)']
def create_bar2(df,threshold,bar):
  Cum_currency = 0
  Cum_BTC = 0
  Current_num = 0
  Volume_df = pd.DataFrame(columns=df.columns)
  progress_i = 0
  for index,row in df.iterrows():
    #Volumeがリセットされた次の行での処理
    if (Cum_currency == 0) | (Cum_BTC == 0) :
      open_time = index
      open_price = row[0]
      high_price = row[1]
      low_price = row[2]
    
    #前の行までの累積の値
    prev_Cum_currency = Cum_currency
    prev_Cum_BTC = Cum_BTC

    #今回の行を足し合わせる
    Cum_currency += row[5] #Volume_(Currency)
    Cum_BTC += row[4] #Volume_(BTC)

    # 〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜
    #足し合わせた結果閾値から遠のいてしまった場合
    #前の行までで区切りをつけて、今処理している行を新しい行とする
    if (abs(threshold - prev_Cum_currency) < abs(threshold- Cum_currency)) & (bar == 'Currency'):
      #区切りをつける
      Volume_df.loc[open_time] = [open_price, high_price, low_price, prev_close, prev_Cum_BTC, prev_Cum_currency]

      #今回の行を新しい行として処理
      Cum_currency = row[5]
      Cum_BTC = row[4]
      open_time = index
      open_price = row[0]
      high_price = row[1]
      low_price = row[2]

      #次の行にとってのprevだからprev_close
      prev_close = row[3]
      progress_i += 1
      continue

    if (abs(threshold - prev_Cum_BTC) < abs(threshold- Cum_BTC)) & (bar == 'BTC'):
      #区切りをつける
      Volume_df.loc[open_time] = [open_price, high_price, low_price, prev_close, prev_Cum_BTC, prev_Cum_currency]

      #今回の行を新しい行として処理
      Cum_currency = row[5]
      Cum_BTC = row[4]
      open_time = index
      open_price = row[0]
      high_price = row[1]
      low_price = row[2]

      #次の行にとってのprevだからprev_close
      prev_close = row[3]
      progress_i += 1
      continue

    # 〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜〜

    if row[1] > high_price:
      high_price = row[1]
    if row[2] < low_price:
      low_price = row[2]

    prev_close = row[3]
    
    #Volume_barの時
    if (Cum_BTC > threshold) & (bar == 'BTC'):
      close_price = row[3] #close
      Volume_df.loc[open_time] = [open_price, high_price, low_price, close_price, Cum_BTC, Cum_currency]
      Cum_currency = 0
      Cum_BTC = 0

    #ドルバーの時
    if (Cum_currency > threshold) & (bar == 'Currency'):
      close_price = row[3] #close
      Volume_df.loc[open_time] = [open_price, high_price, low_price, close_price, Cum_BTC, Cum_currency]
      Cum_currency = 0
      Cum_BTC = 0
    
    progress_i += 1
  
  return Volume_df

In [ ]:
BAR = 'BTC'
Threshold = 200
Volume_df = create_bar2(data,Threshold,BAR)
Volume_df.head()